# Flask-SQLAlchemy

Flask itself does not come with any database backend, but the Flask-SQLAlchemy plugin provides database access via the SQLAlchemy database toolkit.



In [1]:
!pip install flask-sqlalchemy

Looking in links: /home/rick446/src/wheelhouse
You should consider upgrading via the '/home/rick446/.virtualenvs/classes/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.append('./data/flask-examples')

In [3]:
%%file data/flask-examples/sa_model.py
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    postedDate = db.Column(db.DateTime)
    authorName = db.Column(db.String)
    title = db.Column(db.String)
    content = db.Column(db.String)
    
class Comment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.id'), nullable=False)
    postedDate = db.Column(db.DateTime)
    authorName = db.Column(db.String)
    content = db.Column(db.String)
    
    post = db.relationship('Post', backref=db.backref('comments', lazy=True))


Overwriting data/flask-examples/sa_model.py


In [4]:
%%file data/flask-examples/sa-app.py
from datetime import datetime

from flask import Flask, url_for, jsonify, request, abort

from .sa_model import db, Post, Comment

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
#app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)
# db = SQLAlchemy(app)


@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts', _external=True)})


@app.route('/post')
def get_posts():
    post_links = [
        url_for('get_post', post_id=post.id, _external=True) 
        for post in Post.query]
    return jsonify(
        _links={'self': url_for('get_posts', _external=True)},
        posts=[dict(_links=dict(self=link)) for link in post_links])

@app.route('/post', methods=['POST'])
def create_post():
    post = Post(
        authorName=request.authorization.username,
        postedDate=datetime.utcnow(),
        title=request.json['title'],
        content=request.json['content'],
    )
    db.session.add(post)
    db.session.commit()
    result = jsonify_post(post)
    result.headers['Location'] = url_for('get_post', post_id=post.id, _external=True)
    return result, 201

@app.route('/post/<int:post_id>')
def get_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    return jsonify_post(post)

@app.route('/post/<int:post_id>', methods=['PUT'])
def update_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    post.authorName = request.authorization.username
    post.postedDate = datetime.utcnow()
    post.title = request.json['title']
    post.content = request.json['content']
    db.session.commit()
    return jsonify_post(post)

@app.route('/post/<int:post_id>', methods=['DELETE'])
def delete_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    db.session.delete(post)
    db.session.commit()
    return '', 204

def jsonify_post(post):
    return jsonify(
        _links={'self': url_for('get_post', post_id=post.id, _external=True)},
        postedDate=post.postedDate.isoformat(),
        authorName=post.authorName,
        title=post.title,
        content=post.content,
    )

Overwriting data/flask-examples/sa-app.py


In [5]:
import sqlite3
conn = sqlite3.connect('data/flask-examples/blog.db')
with conn as c:
    c.execute('DELETE FROM comment')
    c.execute('DELETE FROM post')



In [6]:
import requests
sess = requests.Session()
sess.auth = ('rick', 'password')

In [16]:
resp = sess.post('http://localhost:5012/post', json={
    'title': 'First post!',
    'content': 'This is the first post, does it work?',
})
resp

<Response [201]>

In [8]:
resp.json()

{'_links': {'self': 'http://localhost:5012/post/1'},
 'authorName': 'rick',
 'content': 'This is the first post, does it work?',
 'postedDate': '2021-11-03T15:40:56.258415',
 'title': 'First post!'}

In [9]:
resp.headers

{'Content-Type': 'application/json', 'Content-Length': '185', 'Location': 'http://localhost:5012/post/1', 'Server': 'Werkzeug/2.0.1 Python/3.8.5', 'Date': 'Wed, 03 Nov 2021 15:40:56 GMT'}

In [10]:
for row in conn.execute('SELECT * FROM post'):
    print(row)

(1, '2021-11-03 15:40:56.258415', 'rick', 'First post!', 'This is the first post, does it work?')


In [11]:
sess.get('http://localhost:5012').json()

{'_links': {'posts': 'http://localhost:5012/post'}}

In [18]:
resp = sess.get('http://localhost:5012/post')
resp.raise_for_status()
resp.json()

{'_links': {'self': 'http://localhost:5012/post'},
 'posts': [{'_links': {'self': 'http://localhost:5012/post/1'}},
  {'_links': {'self': 'http://localhost:5012/post/2'}},
  {'_links': {'self': 'http://localhost:5012/post/3'}},
  {'_links': {'self': 'http://localhost:5012/post/4'}}]}

In [19]:
for post in resp.json()['posts']:
    print('Post')
    r = sess.get(post['_links']['self'])
    r.raise_for_status()
    print(r.json())

Post
{'_links': {'self': 'http://localhost:5012/post/1'}, 'authorName': 'rick', 'content': 'This is the first post, does it work?', 'postedDate': '2021-11-03T15:40:56.258415', 'title': 'First post!'}
Post
{'_links': {'self': 'http://localhost:5012/post/2'}, 'authorName': 'rick', 'content': 'This is the first post, does it work?', 'postedDate': '2021-11-03T15:43:12.271584', 'title': 'First post!'}
Post
{'_links': {'self': 'http://localhost:5012/post/3'}, 'authorName': 'rick', 'content': 'This is the first post, does it work?', 'postedDate': '2021-11-03T15:43:13.144921', 'title': 'First post!'}
Post
{'_links': {'self': 'http://localhost:5012/post/4'}, 'authorName': 'rick', 'content': 'This is the first post, does it work?', 'postedDate': '2021-11-03T15:43:13.939265', 'title': 'First post!'}


In [20]:
for post in resp.json()['posts']:
    print(sess.delete(post['_links']['self']))

<Response [204]>
<Response [204]>
<Response [204]>
<Response [204]>


In [21]:
for row in conn.execute('SELECT * FROM post'):
    print(row)

Open the [Flask-SQLAlchemy lab](./flask-sqlalchemy-lab.ipynb)

In [22]:
from flask import g

In [ ]:
g.user = determine_user_from_request(request)